In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-

# =============================================================================
# MODULES IMPORT
# =============================================================================
from __future__ import print_function
import sys
import os
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
import scipy as sc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.mixture import GaussianMixture
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
import pickle
from sklearn import preprocessing
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import RepeatedStratifiedKFold


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

def getFileList(root):
    import os
    l =[]
    for path, dirs, files in os.walk(root):
         [l.append(os.path.join(path, f)) for f in files]
    return l


def sortTarget(im_folder, im_info):
    import numpy as np

    targets = np.empty(shape=im_folder.shape)
    idx = 0
    for filename in im_folder:
        targets[idx] = np.where(im_info.filename == filename.split('/')[-1])[0].astype(int)
        idx+=1

    return im_info.loc[targets,:]


def ReadImage(filename):
    import cv2
    return cv2.imread(filename)

In [ ]:
# source = https://youtu.be/yUrwEYgZUsA

def normaalize(image):


    img=cv2.imread(image, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    Io = 240 # Transmitted light intensity, Normalizing factor for image intensities
    alpha = 1  #As recommend in the paper. tolerance for the pseudo-min and pseudo-max (default: 1)
    beta = 0.15 #As recommended in the paper. OD threshold for transparent pixels (default: 0.15)


    ######## Step 1: Convert RGB to OD ###################
    ## reference H&E OD matrix.
    #Can be updated if you know the best values for your image.
    #Otherwise use the following default values.
    #Read the above referenced papers on this topic.
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
    ### reference maximum stain concentrations for H&E
    maxCRef = np.array([1.9705, 1.0308])


    # extract the height, width and num of channels of image
    h, w, c = img.shape

    # reshape image to multiple rows and 3 columns.
    #Num of rows depends on the image size (wxh)
    img = img.reshape((-1,3))

    # calculate optical density
    # OD = −log10(I)
    #OD = -np.log10(img+0.004)  #Use this when reading images with skimage
    #Adding 0.004 just to avoid log of zero.

    OD = -np.log10((img.astype(np.float)+1)/Io) #Use this for opencv imread
    #Add 1 in case any pixels in the image have a value of 0 (log 0 is indeterminate)

    """
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.scatter(img[:,0],img[:,1],img[:,2])
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.scatter(OD[:,0],OD[:,1],OD[:,2])
    plt.show()
    """

    ############ Step 2: Remove data with OD intensity less than β ############
    # remove transparent pixels (clear region with no tissue)
    ODhat = OD[~np.any(OD < beta, axis=1)] #Returns an array where OD values are above beta
    #Check by printing ODhat.min()

    ############# Step 3: Calculate SVD on the OD tuples ######################
    #Estimate covariance matrix of ODhat (transposed)
    # and then compute eigen values & eigenvectors.
    eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))


    ######## Step 4: Create plane from the SVD directions with two largest values ######
    #project on the plane spanned by the eigenvectors corresponding to the two
    # largest eigenvalues
    That = ODhat.dot(eigvecs[:,1:3]) #Dot product

    ############### Step 5: Project data onto the plane, and normalize to unit length ###########
    ############## Step 6: Calculate angle of each point wrt the first SVD direction ########
    #find the min and max vectors and project back to OD space
    phi = np.arctan2(That[:,1],That[:,0])

    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100-alpha)

    vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)


    # a heuristic to make the vector corresponding to hematoxylin first and the
    # one corresponding to eosin second
    if vMin[0] > vMax[0]:
        HE = np.array((vMin[:,0], vMax[:,0])).T

    else:
        HE = np.array((vMax[:,0], vMin[:,0])).T


    # rows correspond to channels (RGB), columns to OD values
    Y = np.reshape(OD, (-1, 3)).T

    # determine concentrations of the individual stains
    C = np.linalg.lstsq(HE,Y, rcond=None)[0]

    # normalize stain concentrations
    maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
    tmp = np.divide(maxC,maxCRef)
    C2 = np.divide(C,tmp[:, np.newaxis])

    ###### Step 8: Convert extreme values back to OD space
    # recreate the normalized image using reference mixing matrix

    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm>255] = 254
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)

    # Separating H and E components

    H = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,0], axis=1).dot(np.expand_dims(C2[0,:], axis=0))))
    H[H>255] = 254
    H = np.reshape(H.T, (h, w, 3)).astype(np.uint8)

    E = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,1], axis=1).dot(np.expand_dims(C2[1,:], axis=0))))
    E[E>255] = 254
    E = np.reshape(E.T, (h, w, 3)).astype(np.uint8)

    return Inorm

In [ ]:
def show_sift_features(gray_img, color_img, kp):
    return plt.imshow(cv2.drawKeypoints(gray_img, kp, color_img.copy()))

   # kpimg = cv2.drawKeypoints(im_gray, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
   # plt.imshow(kpimg)

def computeFV(xx, gmm):
    """Computes the Fisher vector on a set of descriptors.
    Parameters
    ----------
    xx: array_like, shape (N, D) or (D, )
        The set of descriptors
    gmm: instance of sklearn mixture.GMM object
        Gauassian mixture model of the descriptors.
    Returns
    -------
    fv: array_like, shape (K + 2 * D * K, )
        Fisher vector (derivatives with respect to the mixing weights, means
        and variances) of the given descriptors.
    Reference
    ---------
    J. Krapac, J. Verbeek, F. Jurie.  Modeling Spatial Layout with Fisher
    Vectors for Image Categorization.  In ICCV, 2011.
    http://hal.inria.fr/docs/00/61/94/03/PDF/final.r1.pdf
    """
    xx = np.atleast_2d(xx)
    N = xx.shape[0]

    # Compute posterior probabilities.
    Q = gmm.predict_proba(xx)  # NxK

    # Compute the sufficient statistics of descriptors.
    Q_sum = np.sum(Q, 0)[:, np.newaxis] / N
    Q_xx = np.dot(Q.T, xx) / N
    Q_xx_2 = np.dot(Q.T, xx ** 2) / N

    # Compute derivatives with respect to mixing weights, means and variances.
    d_pi = Q_sum.squeeze() - gmm.weights_
    d_mu = Q_xx - Q_sum * gmm.means_
    d_sigma = (
            - Q_xx_2
            - Q_sum * gmm.means_ ** 2
            + Q_sum * gmm.covariances_
            + 2 * Q_xx * gmm.means_)

    # Merge derivatives into a vector.
    return np.hstack((d_pi, d_mu.flatten(), d_sigma.flatten()))



def FeatureExtract(im_file, nkeys, pca, gmm, scaler):
    # read image
    im = normaalize(im_file)

    # rgb normalization
   # im = rgb_normalized(im)

    # to gray
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    #im_gray = cv2.normalize(im_gray,im_gray, 0, 255, cv2.NORM_MINMAX)

    # apply gaussian filter
    #im_gray = cv2.GaussianBlur(im_gray,(15,15),0)

    # extract SIFT descriptors
    orb = cv2.ORB_create(nfeatures = nkeys)
    kp, descriptors = orb.detectAndCompute(im_gray, None)
    descriptors = np.array(descriptors, dtype=float)

    descriptors /= (descriptors.sum(axis=1, keepdims=True) + 1e-7)
    descriptors = np.sqrt(descriptors)

    # apply scaler and pca transform
    descriptors = scaler.transform(descriptors)
    descriptors = pca.transform(descriptors)

    # compute Fisher Vector
    fv = computeFV(descriptors, gmm)

    # power-normalization
   # fv = np.sign(fv) * np.abs(fv) ** 0.5
    # L2 normalize
    #fv /= np.sqrt(np.sum(fv ** 2))

    return fv

In [ ]:
#==============================================================================
# IMAGE PARAMETERS
#==============================================================================
path_train = '/content/drive/MyDrive/BC_IDC/4x/train_uncat_4x'
path_test = '/content/drive/MyDrive/BC_IDC/4x/test_uncat_4x'

test_perc = 0.2 # test set percentage

#==============================================================================
# GET IMAGE LIST AND INFO
#==============================================================================
im_folder_train = np.array(getFileList(path_train)) # image list
im_folder_test = np.array(getFileList(path_test))

# Load csv with image information
im_info_train = pd.read_csv('/content/drive/MyDrive/BC_IDC/4x/train_labels_4x.csv')
im_info_test = pd.read_csv('/content/drive/MyDrive/BC_IDC/4x/test_labels_4x.csv')



# =============================================================================
# MATCH IMAGE LIST AND LABELS
# =============================================================================

im_info_train = sortTarget(im_folder_train,im_info_train)
im_info_test = sortTarget(im_folder_test,im_info_test)

le = preprocessing.LabelEncoder()
T_train = im_info_train.target
T_train = np.array(le.fit_transform(T_train))

T_test = im_info_test.target
T_test = np.array(le.fit_transform(T_test))


In [ ]:
# =============================================================================
# TRAIN/TEST SPLIT
# =============================================================================

y_train = T_train
y_test = T_test


print('Train set size', y_train.shape[0], 'images')
print('Test set size', y_test.shape[0], 'images')
print(y_test)
print(y_train)

In [ ]:
# =============================================================================
# FISHER VECTOR PARAMETERS
# =============================================================================
n_cmp = 10 # pca components
k = 256 # gmm n centroids
fnum = 8192 # n sift descriptors

In [ ]:
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

In [ ]:
# ============================================================================
# EXTRACT SIFT DESCRIPTORS FROM TRAIN SET
# =============================================================================
dictionary = []
for file  in tqdm(im_folder_train):
    im = normaalize(file)
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    # Extract sift descriptors
    orb = cv2.ORB_create(nfeatures = fnum)
    kp, descriptors = orb.detectAndCompute(im_gray, None)
    descriptors = np.array(descriptors, dtype=float)
    descriptors /= (descriptors.sum(axis=1, keepdims=True) + 1e-7)
    descriptors = np.sqrt(descriptors)

    dictionary.append(descriptors)


dictionary = np.asarray(dictionary)
dictionary = np.concatenate(dictionary).astype(None)
print(descriptors)
print(dictionary)

In [ ]:

#with open("/content/drive/MyDrive/BreastCancer-Classifier/descrip_norm.pkl", 'wb') as d:

    #pickle.dump(descriptors, d, protocol=pickle.HIGHEST_PROTOCOL)

#with open("/content/drive/MyDrive/BreastCancer-Classifier/descrip.pkl", 'rb') as d1:
    #descriptors = pickle.load(d1)

#print(descriptors)

In [ ]:



#with open("/content/drive/MyDrive/BreastCancer-Classifier/dict_norm_E_H.pkl", 'wb') as f:
    # indent=2 is not needed but makes the file human-readable
    #pickle.dump(dictionary, f, protocol=pickle.HIGHEST_PROTOCOL)

#with open("/content/drive/MyDrive/BreastCancer-Classifier/dict.pkl", 'rb') as f1:
    #dictionary = pickle.load(f1)

print(dictionary.shape)

In [ ]:
# =============================================================================
# APPLY PCA TO DESCRIPTORS LIBRARY
# =============================================================================
sift_scaler = preprocessing.StandardScaler()
descriptors = sift_scaler.fit_transform(descriptors)

sift_pca = PCA(n_components=n_cmp,whiten=True)
dictionary = sift_pca.fit_transform(dictionary)
dictionary = np.float32(dictionary)

with open('/content/drive/MyDrive/BC_IDC/4x/pca_transform_norm.pickle', 'wb') as handle:
    pickle.dump(sift_pca, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/MyDrive/BC_IDC/4x/scaler_norm.pickle', 'wb') as handle:
    pickle.dump(sift_scaler, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open("/content/drive/MyDrive/BC_IDC/4x/my_dict_norm_H.pkl", 'wb') as f:

    pickle.dump(dictionary, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/content/drive/MyDrive/40x/pca_transform_norm.pickle', 'rb') as handle:
    sift_pca = pickle.load(handle)
with open('/content/drive/MyDrive/40x/scaler_norm.pickle', 'rb') as handle:
    sift_scaler = pickle.load(handle)

with open("/content/drive/MyDrive/40x/my_dict_norm_H.pkl", 'rb') as f:
    # indent=2 is not needed but makes the file human-readable
    dictionary = pickle.load(f)

In [ ]:
## =============================================================================
## BUILD DICTIONARY MODEL
## =============================================================================
gmm_pca = GaussianMixture(n_components = k, covariance_type = "diag").fit(dictionary)
with open("/content/drive/MyDrive/BC_IDC/4x/gmm_norm256_H.pkl", 'wb') as f:

    pickle.dump(gmm_pca, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:

#with open('pca4.pickle', 'rb') as handle:
#    sift_pca = pickle.load(handle)
#with open('scaler4.pickle', 'rb') as handle:
#    sift_scaler = pickle.load(handle)

In [ ]:
#with open("/content/drive/MyDrive/BreastCancer-Classifier/gmm_norm256.pkl", 'rb') as f:

    #gmm_pca = pickle.load(f)

In [ ]:
# =============================================================================
 # COMPUTE FISHER VECTORS FOR TRAIN SET
# =============================================================================
X_train = np.empty((y_train.shape[0],k+2*dictionary.shape[1]*k))
idx = 0
for file in tqdm(im_folder_train):
    X_train[idx,:] = FeatureExtract(file, nkeys = fnum, pca = sift_pca, gmm = gmm_pca, scaler = sift_scaler)
    idx += 1

In [ ]:
# =============================================================================
 # COMPUTE FISHER VECTORS FOR TEST SET
# =============================================================================
X_test = np.empty((y_test.shape[0],k+2*dictionary.shape[1]*k))

idx = 0
for file in tqdm(im_folder_test):
    X_test[idx,:] = FeatureExtract(file, nkeys = fnum, pca = sift_pca, gmm = gmm_pca, scaler = sift_scaler)
    idx += 1

In [ ]:
with open('./X_train_norm320H.pickle', 'wb') as handle:
    pickle.dump(X_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./X_test_norm320H.pickle', 'wb') as handle:
    pickle.dump(X_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#with open('/content/drive/MyDrive/BreastCancer-Classifier/X_train.pickle', 'rb') as handle:
    #X_train = pickle.load(handle)
#with open('/content/drive/MyDrive/BreastCancer-Classifier/X_test.pickle', 'rb') as handle:
    #X_test = pickle.load(handle)
print(X_test.shape)
print(X_train.shape)

In [ ]:
# =============================================================================
# PRE PROCESSING
# =============================================================================
#ch2 = SelectKBest(chi2, k=100)
#X_train = ch2.fit_transform(X_train, y_train)
#X_test = ch2.transform(X_test)
#

##PCA
pca = PCA(n_components = 20, whiten=True,random_state=42)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
#
## SCALING data
#scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=True)
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# =============================================================================
# SVM MODEL parameters
# =============================================================================
sss = RepeatedStratifiedKFold(n_splits=5, n_repeats=200, random_state=42)

C_range = 2. ** np.arange(0, 1, step=0.05) # finer search
g_range = np.logspace(-2, -1, 20)

tuned_parameters = [{'kernel': ['rbf'], 'gamma': g_range, 'C': C_range}]
#tuned_parameters = [{'kernel': ['linear'],  'C': C_range}]
#tuned_parameters = [{'kernel': ['poly'],  'C': C_range,'degree': [2,3,4,5,6,7,8]}]


In [ ]:
# ==============================================================================
 # GRID SEARCH
# ==============================================================================
scores = ['f1']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(cache_size=2000, random_state = 42, decision_function_shape='ovr'), tuned_parameters, cv=sss,
                       scoring='%s_macro' % score, n_jobs=-1)

# =============================================================================
#     COMPUTE PARAMETERS
# =============================================================================
    t2 = time.time()
    clf.fit(X_train, y_train)
    elapsed2 = time.time() - t2
    print()
    print("Best parameters set found on development set:")
    print(clf.best_params_)
    print()

    print('Training time: ', elapsed2)


In [ ]:
# EVALUATION
# ==============================================================================
# TRAIN SET
# ===========================================================================
clf2 = clf.best_estimator_
#s print(clf2)
print("Classification on training set:")
y_true, y_pred = y_train, clf2.predict(X_train)
print('Confusion matrix:')
print(confusion_matrix(y_true, y_pred))
print(" Train set f1 score: " + str(f1_score(y_true, y_pred, average='macro')))

In [ ]:
# ==============================================================================
# TESTING
# ==============================================================================
y_true, y_pred = y_test, clf2.predict(X_test)
y_pred_ci = clf.decision_function(X_test)
print("Classification on test set:")
print(classification_report(y_true, y_pred))
print('Confusion matrix:')
print(confusion_matrix(y_true, y_pred))
